In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import sklearn as sk
import sys
import pickle

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from elegir_modelo import metodo
import procesos
import lq

In [13]:
covid = gpd.read_file('datos/covid_periodos.shp', index = True)
covid = covid.set_index(['link','mes']).sort_index(level = 0)
covid = covid.loc[pd.IndexSlice[:,'2020-03':],:]
covid = covid.to_crs('POSGAR94')

# Separamos los campos geometricos del dataframe
geo = covid.loc[pd.IndexSlice[:,'2021-01'],'geometry']
geo = geo.reset_index(level = 'mes', drop = True)
centroides = covid.loc[pd.IndexSlice[:,'2021-01'],'geometry'].to_crs('POSGAR94').centroid
centroides = centroides.reset_index(level = 'mes', drop = True)
print("las cordenadas CRS son: "+str(geo.crs))
codiprov = covid.loc[pd.IndexSlice[:,'2021-01'],['codpcia','departamen','provincia']]


columnas = ['clasificac', 'fallecido']

# Variables acumuladas a partir del mes que todas tienen al menos 1 

covid_acum = covid[columnas].groupby(covid.index.get_level_values(0)).cumsum()
# buscamos el mes en que todos los dptos tienen al menos 1 contagio
mes = 0
valor = True
while valor == True:
    Mes = covid.index.get_level_values(1).unique()[mes]
    valor = np.any(covid_acum.loc[pd.IndexSlice[:,Mes],'clasificac'] == 0)
    mes +=1
print("El mes desde el cuál todos los dptos tienen al menos 1 contagiado es: "+str(Mes))
covid_acum['personas'] = covid.personas

covid2 = covid_acum.loc[pd.IndexSlice[:,Mes:],:]

las cordenadas CRS son: POSGAR94
El mes desde el cuál todos los dptos tienen al menos 1 contagiado es: 2021-01


C:\Users\paban\anaconda3\envs\covid\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  if __name__ == '__main__':


In [32]:
from lq import *
from sklearn.metrics import silhouette_score,  calinski_harabasz_score

lqcovid = lq_peri(covid_acum[['fallecido','personas']])
lqcovid2 = lq_peri(covid_acum[['clasificac','personas']])
lqcovid3 = lq_peri(covid2[['fallecido','clasificac']])

def df_datos(df):
    mes = ['2020-03']
    df = df.copy().set_index('link')
    col = list(df.columns)
    ind = col.index('personas')
    return df[col[:ind]].copy()

def siluetas(X, model):
    X_ = df_datos(X)
    return silhouette_score(X_, X[model])
def calinski(X, model):
    X_ = df_datos(X)
    return sk.metrics.calinski_harabasz_score(X_, X[model])
def inv_davies_bouldin(X,model):
    X_ = df_datos(X)
    return (sk.metrics.davies_bouldin_score(X_,X[model]))**-1
def indice_lq(X, model):
    X_ = df_datos(X)
    grupos = X[model].values
    ind = lqcovid.calcular_indice_debil(grupos)
    return 1-ind

In [36]:
archivo = 'df_R/fallec.shp'
df =gpd.read_file('df_R/fallec.shp')
#df = df.set_index(df.link.astype(int))
#df = df.drop('link', axis = 1)
[[siluetas(df,'sk'),siluetas(df,'redcap'),siluetas(df,'schc')],
[indice_lq(df,'sk'),indice_lq(df,'redcap'),indice_lq(df,'schc')]]

[[0.18425467546513144, 0.08774696567418787, -0.01754359092252447],
 [0.5941176470588235, 0.5117647058823529, -0.11764705882352944]]

In [41]:
archivos = ['fallec', 'fallec_pca', 'positiv', 'positiv_pca','fall_pos','fall_pos_pca']
result = {}
for a in archivos:
    arch = 'df_R/'+a+'.shp'
    df = gpd.read_file(arch)
    r = [[siluetas(df,'sk'),siluetas(df,'redcap'),siluetas(df,'schc')],
    [calinski(df,'sk'),calinski(df,'redcap'),calinski(df,'schc')],
    [inv_davies_bouldin(df,'sk'),inv_davies_bouldin(df,'redcap'),inv_davies_bouldin(df,'schc')], 
    [indice_lq(df,'sk'),indice_lq(df,'redcap'),indice_lq(df,'schc')]]
    r = pd.DataFrame(r, columns = ('sk','redcap','schc'), index = ['sil','cal','dav','lqg'])
    result[a] = r
    

In [44]:
for k in result.keys():
    print(k),
    print(result[k])

fallec
            sk     redcap      schc
sil   0.184255   0.087747 -0.017544
cal  89.383244  92.280739  1.332481
dav   0.400619   0.422436  1.156588
lqg   0.594118   0.511765 -0.117647
fallec_pca
             sk     redcap      schc
sil    0.185413   0.180936 -0.093710
cal  102.356493  98.872051  1.045721
dav    0.397420   0.390431  0.996406
lqg    0.564706   0.564706 -0.094118
positiv
             sk      redcap      schc
sil    0.248795    0.281476 -0.144715
cal  108.527928  123.649623  1.545357
dav    0.725367    0.759896  0.851602
lqg    0.535294    0.529412 -0.041176
positiv_pca
             sk      redcap      schc
sil    0.204395    0.177647 -0.212282
cal  107.584460  106.590642  0.708889
dav    0.600864    0.533063  0.732134
lqg    0.541176    0.641176 -0.129412
fall_pos
            sk      redcap      schc
sil   0.187754    0.191917 -0.005955
cal  88.304911  117.160714  1.558539
dav   0.123380    0.259079  1.236087
lqg   0.470588    0.600000  0.052941
fall_pos_pca
          